In [2]:
import numpy as np
from ase.io import read, write
from ase.visualize import view
from ase.data.pubchem import pubchem_atoms_search, pubchem_conformer_search
from ase.build import add_adsorbate
from ase.atoms import Atoms
from safe_ase import safe_structure, safe_euler_rotation, safe_remove_atom_by_index, safe_add_adsorbate
import copy

In [49]:
dmso = pubchem_atoms_search(cid=679)
acn = pubchem_atoms_search(cid=6342)

In [52]:
view(safe_euler_rotation(acn,90,0,0))

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [53]:
acn = safe_euler_rotation(acn,90,0,0)

In [55]:
write('acn.cif', acn)

In [56]:
view(read('acn.cif'))

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [17]:
mono = read('mono4.cif')

In [11]:
view(mono*(2,2,1) + dmso)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [4]:
def set_to_origin(arr):
    arr[:,0] -= arr[:,0].min()
    arr[:,1] -= arr[:,1].min()
    arr[:,2] -= arr[:,2].min()
    return arr

In [35]:
def center_by_index(arr, idx):
    arr[:,0] -= arr[idx,0]
    arr[:,1] -= arr[idx,1]
    arr[:,2] -= arr[idx,2]
    return arr

In [5]:
def move_to(arr, x, y, z):
    arr[:,0] += x
    arr[:,1] += y
    arr[:,2] += z
    return arr

In [9]:
dmso.positions = set_to_origin(dmso.positions)

In [10]:
dmso.positions = move_to(dmso.positions, 0.72252, 1.118795, 14)

In [38]:
view(ed)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [18]:
view(mono*(2,2,1))

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [16]:
mono21 = mono*(2,2,1)

NameError: name 'mono' is not defined

In [78]:
mono21.cell

Cell([6.64404, 4.56659, 22.47])

In [6]:
def freeze_atoms(structure: Atoms, freeze_index = []) -> np.array:
    cons = np.ones((len(structure),3), dtype=int)
    for f in freeze_index:
        cons[f] = 0
    return cons

In [7]:
def pretty_coordinates(structure: Atoms, frozen_atoms = []):
    symbols = np.array(structure.get_chemical_symbols())
    symbols.shape = (len(symbols), 1) # Fix dimensions in order to concatenate
    matched = np.concatenate((symbols, structure.get_positions(), freeze_atoms(structure, frozen_atoms)), axis=1)
    # matched = np.concatenate((symbols, structure.get_positions()), axis=1)
    pretty = ['      '.join(coord) for coord in matched]
    return pretty

In [8]:
def pretty_cell(structure: Atoms, distance: float):
    cell = structure.cell
    cell[2] = cell[2] + [0,0,distance]
    axis = [ c for c in cell]
    return [ '      '.join([str(i) for i in a]) for a in axis] # Doble recursion

In [9]:
from jinja2 import Environment, FileSystemLoader

In [22]:
mono22 = mono*(2,2,1)

In [28]:
mono22.positions[:,2]

array([10.1683491, 10.1683491, 12.304572 , 12.304572 , 10.1683491,
       10.1683491, 12.304572 , 12.304572 , 10.1683491, 10.1683491,
       12.304572 , 12.304572 , 10.1683491, 10.1683491, 12.304572 ,
       12.304572 ])

In [36]:
def build_dmso_mono(z_distance):
    mono = read('mono4.cif')
    mono22 = mono*(2,2,1)
    z_max = mono22.positions[:,2].max() # uppermost postion of p layer
    y_max = mono22.positions[:,1].max() 
    x_max = mono22.positions[:,0].max() 
    y_min = mono22.positions[:,1].min() 
    x_min = mono22.positions[:,0].min() 
    dmso = read('dmso.cif')
    dmso.positions = center_by_index(dmso.positions,0)
    delta_y = (y_max - y_min) / 2. + y_min
    delta_x = (x_max - x_min) / 2. + x_min
    dmso.positions = move_to(dmso.positions, delta_x, delta_y, z_max + z_distance)
    both_layers = mono22 + dmso
    both_layers.cell[2,2] = both_layers.positions[:,2].max() + 5
    return both_layers

In [47]:
def gen_dmso_mono_input(template_name: str,
    prefix: str, distance: float,
    frozen_atoms = []
):
    environment = Environment(loader=FileSystemLoader("templates/"))
    template = environment.get_template(template_name)
    both = build_dmso_mono(distance)
    s_distance = "{:.2f}".format(distance)
    input_render = {
    "prefijo": prefix + s_distance,
    "dirSalida": prefix + s_distance,
    "nat": len(both),
    "ntyp": len(set(both.get_chemical_symbols())),
    "coordenadas": pretty_coordinates(both, frozen_atoms),
    "ejes": pretty_cell(both, distance)
    }
    with open(prefix + s_distance + "-" + template_name,'w',encoding = 'utf-8') as f:
        f.write(template.render(input_render))
        f.close()

In [41]:
view(build_dmso_mono(2.0))

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [46]:
algo = build_dmso_mono(2.0)
len(set(algo.get_chemical_symbols()))

5

In [26]:
gen_dmso_mono_input('scf_dmso.in', 'dmso', 2.3,frozen_atoms=range(8))

In [21]:
np.arange(1.5,3.80,0.05)

array([3.25, 3.3 , 3.35, 3.4 , 3.45, 3.5 , 3.55, 3.6 , 3.65, 3.7 , 3.75])

In [48]:
[gen_dmso_mono_input('scf_dmso.in', 'dmso', i, frozen_atoms=range(8)) for i in np.arange(2.00,5.5,0.5)]

[None, None, None, None, None, None, None]

In [42]:
def build_ed_sandwich(z_distance, x = 0.72252, y = 1.18795):
    z_max = 12.305 # uppermost postion of p layer
    delta_ed = 1.770
    delta_intra = 2.137
    lower = read('mono4.cif')
    upper = read('mono4.cif')
    ed = read('ed.cif')
    lower21 = lower*(2,1,1)
    upper21 = upper*(2,1,1)
    ed.positions = set_to_origin(ed.positions)
    ed.positions = move_to(ed.positions, x, y, z_max + z_distance)
    upper21.positions = move_to(upper21.positions, 0, 0, delta_intra + delta_ed + (2 * z_distance))
    three_layers = lower21 + ed + upper21
    three_layers.cell[2,2] = three_layers.positions[:,2].max() + 5
    return three_layers

In [43]:
def gen_ed_sandwich_input(template_name: str,
    prefix: str, distance: float,
    frozen_atoms = []
):
    environment = Environment(loader=FileSystemLoader("templates/"))
    template = environment.get_template(template_name)
    both = build_ed_sandwich(distance)
    s_distance = "{:.2f}".format(distance)
    input_render = {
    "prefijo": prefix + s_distance,
    "dirSalida": prefix + s_distance,
    "nat": len(both),
    "ntyp": len(set(both.get_chemical_symbols())),
    "coordenadas": pretty_coordinates(both, frozen_atoms),
    "ejes": pretty_cell(both, distance)
    }
    with open(prefix + s_distance + "-" + template_name,'w',encoding = 'utf-8') as f:
        f.write(template.render(input_render))
        f.close()

In [47]:
[gen_ed_sandwich_input('scf.in', 'sand', i) for i in np.arange(2.0,3.05,0.05)]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [44]:
gen_ed_sandwich_input('relax.in', 'sand', 2.55)

In [31]:
diamin = read('ed.cif')

In [34]:
view(diamin)

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [32]:
diamin.set_cell((20,20,20))

In [33]:
diamin.positions = move_to(diamin.positions,10,10,10)

In [38]:
mono211 = mono*(2,1,1)

In [40]:
environment = Environment(loader=FileSystemLoader("templates/"))
template = environment.get_template('scf.in')
input_render = {
    "prefijo": 'mono',
    "dirSalida": 'mono',
    "nat": len(mono),
    "coordenadas": pretty_coordinates(mono,  []),
    "ejes": pretty_cell(mono, 0)
    }
with open("mono" + "-" + "scf.in",'w',encoding = 'utf-8') as f:
        f.write(template.render(input_render))
        f.close()

In [2]:
relaxed = read('sand2.55-second-relax.out')

In [4]:
view(relaxed*(3,3,1))

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [5]:
initial = read('sand2.55-second-relax.in')

In [6]:
view(initial*(3,3,1))

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [7]:
initial == relaxed

False

In [12]:
view([initial*(2,2,1), relaxed*(2,2,1)])

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

In [3]:
initial_ed = read('ed.cif')
final_ed = read('ethylendiamine-relax.out')

In [4]:
view([initial_ed,final_ed])

<Popen: returncode: None args: ['/home/roberto/anaconda3/envs/dftb/bin/pytho...>

/home/roberto/anaconda3/envs/dftb/lib/python3.10/site-packages/ase/gui/images.py:123: UserWarning: Not all images have the same boundary conditions!
  warnings.warn('Not all images have the same boundary conditions!')
Traceback (most recent call last):
  File "/home/roberto/anaconda3/envs/dftb/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/roberto/anaconda3/envs/dftb/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/roberto/anaconda3/envs/dftb/lib/python3.10/site-packages/ase/gui/pipe.py", line 32, in <module>
    main()
  File "/home/roberto/anaconda3/envs/dftb/lib/python3.10/site-packages/ase/gui/pipe.py", line 28, in main
    plt.show()
  File "/home/roberto/anaconda3/envs/dftb/lib/python3.10/site-packages/matplotlib/pyplot.py", line 368, in show
    return _backend_mod.show(*args, **kwargs)
  File "/home/roberto/.local/lib/python3.10/site-packages/matplotlib_inline/backe